
# Raster Priming with GDAL/OGR Command Line

**Brek Chiles, 2025**



### Purpose

This notebook provides a workflow with GDAL/OGR command line tools that create raster data inputs for the GeoPandas_Rasterio_Zonal_Stistics IPython Notebook.


In [ ]:
# Import modules
import os

In [ ]:
# Set directory paths
ipynb_dir = os.getcwd()
boundary_dir = os.path.join(ipynb_dir, "Inputs", "Boundary")
dem_dir = os.path.join(ipynb_dir, "Inputs", "DEM")

# Create the DEM directory
os.makedirs(dem_dir, exist_ok=True)

# Check directory paths
ipynb_dir, boundary_dir, dem_dir

In [ ]:
# Set file paths
# Raster
raster_input = os.path.join(dem_dir, "King_Co_2021_DTM_Crop.tif")
raster_output = os.path.join(dem_dir, "King_Co_2021_Ext.tif")
hillshade_output = os.path.join(dem_dir, "King_Co_2021_SR.tif")
slope_output = os.path.join(dem_dir, "King_Co_2021_Ext_Perc.tif")

# Vector/Geometry
redmond_neighborhoods = os.path.join(boundary_dir, "Redmond_Neighborhoods.shp")
boundary = os.path.join(boundary_dir, "Redmond_Study_Area.shp")

# Check directory paths
raster_input, raster_output, hillshade_output, slope_output, redmond_neighborhoods, boundary

In [ ]:
# Set Dropbox Url and directory for input raster files
url = "https://www.dropbox.com/scl/fi/um1idvkj5taiinvsvbcgu/King_Co_2021_DTM_Crop.tif?rlkey=ziqgqu8lmhywilqiwcgtk9e1z&st=24nyyc36&dl=1"


### Raster Extraction by Mask

The following cells will download the supplemental raster GeoTiff and extract data to the study area boundary.


In [ ]:
!curl -L "$url"  -O --output-dir "$dem_dir"


Inspect raster metadata with gdalinfo.


In [ ]:
!gdalinfo $raster_input


Inspect vector or geometric metadata with ogrlinfo. Check out Ogr2Ogr.


In [ ]:
!ogrinfo -al $redmond_neighborhoods

Dissolve redmond_neighborhoods to make boundary.

In [ ]:
!ogr2ogr -f "ESRI Shapefile" $boundary $redmond_neighborhoods -dialect sqlite -sql "SELECT ST_Union(geometry), NAME FROM 'Redmond_Neighborhoods' GROUP BY CreatedBy"


Use gdalwarp to extract raster data to a vector or geometry file.


In [ ]:
!gdalwarp -cutline $boundary -crop_to_cutline $raster_input $raster_output -overwrite


### Raster Processing with gdaldem

GDAL/OGR can analyze DEM data with gdaldem.




Create a Hillshade/Shaded Relief map.


In [ ]:
!gdaldem hillshade -az 275 $raster_input $hillshade_output


Create a slope map measured in percent (-p).


In [ ]:
!gdaldem slope -p $raster_output $slope_output